CODE FOR BJ WHOLESALE EXAMPLE OF INCORRECT IMPLIED SHARE PRICE

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker
ticker = "COST"
bj = yf.Ticker(ticker)

# Function to normalize data (from previous response)
def normalize_data(df):
    df = df.T
    df.index = pd.to_datetime(df.index).year
    df = df / 1_000_000  # Convert to millions
    return df

# 1. Fetch and Normalize Financial Statements
income_stmt = bj.financials
balance_sheet = bj.balance_sheet
cash_flow = bj.cash_flow

income_stmt_normalized = normalize_data(income_stmt)
balance_sheet_normalized = normalize_data(balance_sheet)
cash_flow_normalized = normalize_data(cash_flow)

# Define years dynamically based on latest data
# Get the latest 3 years from income_stmt_normalized
available_years = sorted(income_stmt_normalized.index.tolist(), reverse=True)  # Sort descending to get latest
given_years = available_years[:3]  # Take the latest 3 years
latest_year = given_years[0]  # Most recent year
estimate_years = [latest_year + i for i in range(1, 6)]  # Next 5 years after the latest

# Sort years in ascending order
given_years = sorted(given_years)  # Ascending order for given years
estimate_years = sorted(estimate_years)  # Ascending order for estimate years
all_years = sorted(given_years + estimate_years)  # Ascending order for all years
lowest_given_year = given_years[0]  # First year in ascending order (e.g., 2023)
prior_year = lowest_given_year - 1

# 2. Create Free Cash Flow Table (initially just for components provided)
fcf_df = pd.DataFrame(index=all_years)

# Populate Given Years (2023-2025)
for year in given_years:
    # Map to normalized data
    fcf_df.loc[year, "Revenue"] = income_stmt_normalized.loc[year, "Total Revenue"]
    fcf_df.loc[year, "COGS"] = income_stmt_normalized.loc[year, "Cost Of Revenue"]
    fcf_df.loc[year, "Gross Profit"] = fcf_df.loc[year, "Revenue"] - fcf_df.loc[year, "COGS"]
    fcf_df.loc[year, "SG&A"] = income_stmt_normalized.loc[year, "Selling General And Administration"]
    fcf_df.loc[year, "Total Operating Expenses"] = fcf_df.loc[year, "SG&A"]
    fcf_df.loc[year, "EBITDA"] = fcf_df.loc[year, "Gross Profit"] - fcf_df.loc[year, "Total Operating Expenses"]
    fcf_df.loc[year, "D&A"] = income_stmt_normalized.loc[year, "Reconciled Depreciation"]
    #fcf_df.loc[year, "EBIT"] = fcf_df.loc[year, "EBITDA"] - fcf_df.loc[year, "D&A"]
    fcf_df.loc[given_years, "EBIT"] = income_stmt_normalized.loc[given_years, "Operating Income"]
    #fcf_df.loc[year, "Operating Taxes"] = income_stmt_normalized.loc[year, "Tax Provision"]
    fcf_df.loc[given_years, "Operating Taxes"] = fcf_df.loc[year, "EBIT"] * 0.21
    fcf_df.loc[year, "NOPAT"] = fcf_df.loc[year, "EBIT"] - fcf_df.loc[year, "Operating Taxes"]
    #fcf_df.loc[given_years, "NOPAT"] = fcf_df.loc[year, "EBIT"] * (1 - 0.21)

# 3. Assumptions for Given Years (2023-2025)
fcf_df["Revenue Growth"] = ((fcf_df.loc[given_years, "Revenue"] / fcf_df.loc[given_years, "Revenue"].shift(1)) - 1).fillna(0)
fcf_df["COGS% of Revenue"] = fcf_df.loc[given_years, "COGS"] / fcf_df.loc[given_years, "Revenue"]#asdfasdfasdf
fcf_df["SG&A% of Revenue"] = fcf_df.loc[given_years, "SG&A"] / fcf_df.loc[given_years, "Revenue"] #asdfasdfasdf
fcf_df["Tax% of EBIT"] = fcf_df.loc[given_years, "Operating Taxes"] / fcf_df.loc[given_years, "EBIT"] #asdfasdfads

# Calculate averages for estimates
avg_revenue_growth = fcf_df.loc[given_years, "Revenue Growth"].mean()
avg_cogs_pct = fcf_df.loc[given_years, "COGS% of Revenue"].mean()
avg_sga_pct = fcf_df.loc[given_years, "SG&A% of Revenue"].mean()
avg_tax_pct = fcf_df.loc[given_years, "Tax% of EBIT"].mean()

# Adjust revenue growth for estimates (±2% around mean)
revenue_growth_est = avg_revenue_growth + np.random.uniform(-0.02, 0.02)

# 4. Estimate Values for 2026-2030
for year in estimate_years:
    fcf_df.loc[year, "Revenue"] = fcf_df.loc[year-1, "Revenue"] * (1 + revenue_growth_est)
    fcf_df.loc[year, "COGS"] = fcf_df.loc[year, "Revenue"] * avg_cogs_pct
    fcf_df.loc[year, "Gross Profit"] = fcf_df.loc[year, "Revenue"] - fcf_df.loc[year, "COGS"]
    fcf_df.loc[year, "SG&A"] = fcf_df.loc[year, "Revenue"] * avg_sga_pct
    fcf_df.loc[year, "Total Operating Expenses"] = fcf_df.loc[year, "SG&A"]
    fcf_df.loc[year, "EBITDA"] = fcf_df.loc[year, "Gross Profit"] - fcf_df.loc[year, "Total Operating Expenses"]
    # D&A, EBIT, Operating Taxes, and NOPAT will be added later after verification

# 5. Display the Table
print("Free Cash Flow Table (in millions):\n")
print(fcf_df[["Revenue", "COGS", "Gross Profit", "SG&A", "Total Operating Expenses", "EBITDA",
              "D&A", "EBIT", "Operating Taxes", "NOPAT"]])
print("\nAssumptions for Given Years:\n")
print(fcf_df[["Revenue Growth", "COGS% of Revenue", "SG&A% of Revenue", "Tax% of EBIT"]])

# Save to CSV for reference
fcf_df.to_csv("BJ_Free_Cash_Flow_Table_Partial.csv")
print("\nSaved partial Free Cash Flow table as 'BJ_Free_Cash_Flow_Table_Partial.csv'.")

Free Cash Flow Table (in millions):

            Revenue           COGS  Gross Profit          SG&A  \
2022  226954.000000  199382.000000  27572.000000  19779.000000   
2023  242290.000000  212586.000000  29704.000000  21590.000000   
2024  254453.000000  222358.000000  32095.000000  22810.000000   
2025  261905.867218  229585.166327  32320.700892  23213.693616   
2026  269577.027127  236309.660673  33267.366453  23893.617123   
2027  277472.873465  243231.113843  34241.759622  24593.455427   
2028  285599.987245  250355.294713  35244.692532  25313.791826   
2029  293965.142234  257688.141128  36277.001106  26055.226705   

      Total Operating Expenses        EBITDA     D&A    EBIT  Operating Taxes  \
2022              19779.000000   7793.000000  1900.0  7793.0          1949.85   
2023              21590.000000   8114.000000  2077.0  8114.0          1949.85   
2024              22810.000000   9285.000000  2237.0  9285.0          1949.85   
2025              23213.693616   9107.007275

In [ ]:

# Create Fixed Assets Table
fixed_assets_df = pd.DataFrame(index=all_years)

# Populate Given Years (2023-2025)
for year in given_years:
    fixed_assets_df.loc[year, "Beginning PPE"] = (
        balance_sheet_normalized.loc[prior_year, "Net PPE"] if year == lowest_given_year
        else fixed_assets_df.loc[year-1, "Ending PPE"]
    )
    fixed_assets_df.loc[year, "D&A"] = income_stmt_normalized.loc[year, "Reconciled Depreciation"]
    fixed_assets_df.loc[year, "Ending PPE"] = balance_sheet_normalized.loc[year, "Net PPE"]
    """fixed_assets_df.loc[year, "CapEx"] = (
        fixed_assets_df.loc[year, "Ending PPE"] -
        fixed_assets_df.loc[year, "Beginning PPE"] +
        fixed_assets_df.loc[year, "D&A"]
    )"""
    fixed_assets_df.loc[year, "CapEx"] = cash_flow_normalized.loc[year, "Capital Expenditure"] * -1
    fixed_assets_df.loc[year, "Yahoo CapEx"] = cash_flow_normalized.loc[year, "Capital Expenditure"] * -1  # Convert to positive

# Assumptions for Given Years
fixed_assets_df["D&A% of Beginning PPE"] = fixed_assets_df.loc[given_years, "D&A"] / fixed_assets_df.loc[given_years, "Beginning PPE"]  #asdfasdfasf
fixed_assets_df["CapEx% of Beginning PPE"] = fixed_assets_df.loc[given_years, "CapEx"] / fixed_assets_df.loc[given_years, "Beginning PPE"]  #asdfasdf

avg_da_pct = fixed_assets_df.loc[given_years, "D&A% of Beginning PPE"].mean()
avg_capex_pct = fixed_assets_df.loc[given_years, "CapEx% of Beginning PPE"].mean()

# Estimate Years (2026-2030)
for year in estimate_years:
    fixed_assets_df.loc[year, "Beginning PPE"] = fixed_assets_df.loc[year-1, "Ending PPE"]
    fixed_assets_df.loc[year, "D&A"] = fixed_assets_df.loc[year, "Beginning PPE"] * avg_da_pct
    #fixed_assets_df.loc[year, "CapEx"] = fixed_assets_df.loc[year, "Beginning PPE"] * avg_capex_pct
    fixed_assets_df.loc[year, "CapEx"] = fcf_df.loc[year, "Revenue"] * 0.025
    fixed_assets_df.loc[year, "Ending PPE"] = (
        fixed_assets_df.loc[year, "Beginning PPE"] -
        fixed_assets_df.loc[year, "D&A"] +
        fixed_assets_df.loc[year, "CapEx"]
    )
    fixed_assets_df.loc[year, "D&A% of Beginning PPE"] = avg_da_pct
    fixed_assets_df.loc[year, "CapEx% of Beginning PPE"] = avg_capex_pct

# Display the Table
print("Fixed Assets Table (in millions):\n")
print(fixed_assets_df[["Beginning PPE", "D&A", "Ending PPE", "CapEx", "Yahoo CapEx"]])
print("\nAssumptions:\n")
print(fixed_assets_df[["D&A% of Beginning PPE", "CapEx% of Beginning PPE"]])

# Save to CSV
fixed_assets_df.to_csv("BJ_Fixed_Assets_Table.csv")
print("\nSaved Fixed Assets table as 'BJ_Fixed_Assets_Table.csv'.")

Fixed Assets Table (in millions):

      Beginning PPE          D&A    Ending PPE        CapEx  Yahoo CapEx
2022   26382.000000  1900.000000  27420.000000  3891.000000       3891.0
2023   27420.000000  2077.000000  29397.000000  4323.000000       4323.0
2024   29397.000000  2237.000000  31649.000000  4710.000000       4710.0
2025   31649.000000  2361.676130  36009.004016  6721.680146          NaN
2026   36009.004016  2687.023453  40424.429187  7102.448624          NaN
2027   40424.429187  3016.506351  44912.709638  7504.786802          NaN
2028   44912.709638  3351.425774  49491.200423  7929.916558          NaN
2029   49491.200423  3693.076770  54177.252638  8379.128986          NaN

Assumptions:

      D&A% of Beginning PPE  CapEx% of Beginning PPE
2022               0.072019                 0.147487
2023               0.075748                 0.157659
2024               0.076096                 0.160220
2025               0.074621                 0.155122
2026               0.074621 

In [ ]:

# Create NWC Table
nwc_df = pd.DataFrame(index=all_years)

# Copy Revenue and COGS from FCF table (assuming fcf_df exists from earlier)
nwc_df["Revenue"] = fcf_df["Revenue"]
nwc_df["COGS"] = fcf_df["COGS"]

# Populate Given Years (2023-2025)
for year in given_years:
    nwc_df.loc[year, "Accounts Receivable"] = balance_sheet_normalized.loc[year, "Accounts Receivable"]
    nwc_df.loc[year, "Merchandise Inventory"] = balance_sheet_normalized.loc[year, "Finished Goods"]
    nwc_df.loc[year, "Other Current Assets"] = balance_sheet_normalized.loc[year, "Other Current Assets"]

    nwc_df.loc[year, "Accounts Payable"] = balance_sheet_normalized.loc[year, "Accounts Payable"]
    nwc_df.loc[year, "Current Accrued Expenses"] = balance_sheet_normalized.loc[year, "Current Accrued Expenses"]
    nwc_df.loc[year, "Deferred Membership Fees"] = balance_sheet_normalized.loc[year, "Current Deferred Liabilities"]
    nwc_df.loc[year, "Other Current Liabilities"] = balance_sheet_normalized.loc[year, "Other Current Liabilities"]

# Assumptions for Given Years (2023-2025)
nwc_df["DSO"] = (nwc_df.loc[given_years, "Accounts Receivable"] / nwc_df.loc[given_years, "Revenue"]) * 360
nwc_df["DIO"] = (nwc_df.loc[given_years, "Merchandise Inventory"] / nwc_df.loc[given_years, "COGS"]) * 360
nwc_df["DPO"] = (nwc_df.loc[given_years, "Accounts Payable"] / nwc_df.loc[given_years, "COGS"]) * 360

nwc_df["Other Current Assets % of Revenue"] = nwc_df.loc[given_years, "Other Current Assets"] / nwc_df.loc[given_years, "Revenue"]   #asasdgfgdfgafgag
nwc_df["Current Accrued Expenses % of Revenue"] = nwc_df.loc[given_years, "Current Accrued Expenses"] / nwc_df.loc[given_years, "Revenue"]
nwc_df["Deferred Membership Fees % of Revenue"] = nwc_df.loc[given_years, "Deferred Membership Fees"] / nwc_df.loc[given_years, "Revenue"]
nwc_df["Other Current Liabilities % of Revenue"] = nwc_df.loc[given_years, "Other Current Liabilities"] / nwc_df.loc[given_years, "Revenue"]

# Calculate averages for estimates
avg_dso = nwc_df.loc[given_years, "DSO"].mean()
avg_dio = nwc_df.loc[given_years, "DIO"].mean()
avg_dpo = nwc_df.loc[given_years, "DPO"].mean()
avg_oca_pct = nwc_df.loc[given_years, "Other Current Assets % of Revenue"].mean()
avg_cae_pct = nwc_df.loc[given_years, "Current Accrued Expenses % of Revenue"].mean()
avg_dmf_pct = nwc_df.loc[given_years, "Deferred Membership Fees % of Revenue"].mean()
avg_ocl_pct = nwc_df.loc[given_years, "Other Current Liabilities % of Revenue"].mean()

# Estimate Years (2026-2030)
for year in estimate_years:
    nwc_df.loc[year, "Accounts Receivable"] = avg_dso * nwc_df.loc[year, "Revenue"] / 360
    nwc_df.loc[year, "Merchandise Inventory"] = avg_dio * nwc_df.loc[year, "COGS"] / 360
    nwc_df.loc[year, "Other Current Assets"] = avg_oca_pct * nwc_df.loc[year, "Revenue"]

    nwc_df.loc[year, "Accounts Payable"] = avg_dpo * nwc_df.loc[year, "COGS"] / 360
    nwc_df.loc[year, "Current Accrued Expenses"] = avg_cae_pct * nwc_df.loc[year, "Revenue"]
    nwc_df.loc[year, "Deferred Membership Fees"] = avg_dmf_pct * nwc_df.loc[year, "Revenue"]
    nwc_df.loc[year, "Other Current Liabilities"] = avg_ocl_pct * nwc_df.loc[year, "Revenue"]

# Calculate Current Assets and Liabilities for All Years
nwc_df["Current Assets"] = (
    nwc_df["Accounts Receivable"] +
    nwc_df["Merchandise Inventory"] +
    nwc_df["Other Current Assets"]
)
nwc_df["Current Liabilities"] = (
    nwc_df["Accounts Payable"] +
    nwc_df["Current Accrued Expenses"] +
    nwc_df["Deferred Membership Fees"] +
    nwc_df["Other Current Liabilities"]
)

# Calculate Net Working Capital
nwc_df["Net Working Capital"] = nwc_df["Current Assets"] - nwc_df["Current Liabilities"]

# Display the Table
print("Net Working Capital Table (in millions):")
display(nwc_df[["Revenue", "COGS", "Accounts Receivable", "Merchandise Inventory", "Other Current Assets",
                "Accounts Payable", "Current Accrued Expenses", "Deferred Membership Fees", "Other Current Liabilities",
                "Current Assets", "Current Liabilities", "Net Working Capital"]])

print("\nAssumptions:")
display(nwc_df[["DSO", "DIO", "DPO", "Other Current Assets % of Revenue",
                "Current Accrued Expenses % of Revenue", "Deferred Membership Fees % of Revenue",
                "Other Current Liabilities % of Revenue"]])

# Save to CSV
nwc_df.to_csv("BJ_Net_Working_Capital_Table.csv")
print("\nSaved Net Working Capital table as 'BJ_Net_Working_Capital_Table.csv'.")

Net Working Capital Table (in millions):


,Revenue,COGS,Accounts Receivable,Merchandise Inventory,Other Current Assets,Accounts Payable,Current Accrued Expenses,Deferred Membership Fees,Other Current Liabilities,Current Assets,Current Liabilities,Net Working Capital
2022,226954.000000,199382.000000,2241.000000,17907.000000,1499.000000,17848.000000,6292.000000,2174.000000,5611.000000,21647.000000,31925.000000,-10278.000000
2023,242290.000000,212586.000000,2285.000000,16651.000000,1709.000000,17483.000000,6428.000000,2337.000000,6254.000000,20645.000000,32502.000000,-11857.000000
2024,254453.000000,222358.000000,2721.000000,18647.000000,1734.000000,19421.000000,7229.000000,2501.000000,6210.000000,23102.000000,35361.000000,-12259.000000
2025,268867.205857,235687.435459,2688.548548,19797.644816,1834.840473,20355.333849,7408.531227,2603.837909,6716.339366,24321.033837,37084.042352,-12763.008515
2026,284097.944945,249038.613133,2840.848942,20919.138090,1938.780172,21508.418986,7828.208316,2751.339631,7096.805300,25698.767204,39184.772233,-13486.005029
2027,300191.472087,263146.105817,3001.776820,22104.161504,2048.607828,22726.823874,8271.659193,2907.197003,7498.823796,27154.546152,41404.503867,-14249.957715
2028,317196.662339,278052.757102,3171.820911,23356.313902,2164.656980,24014.248734,8740.230593,3071.883354,7923.615762,28692.791793,43749.978442,-15057.186649
2029,335165.159421,293803.837577,3351.497628,24679.397995,2287.280063,25374.603395,9235.345538,3245.898827,8372.471262,30318.175685,46228.319022,-15910.143336



Assumptions:


,DSO,DIO,DPO,Other Current Assets % of Revenue,Current Accrued Expenses % of Revenue,Deferred Membership Fees % of Revenue,Other Current Liabilities % of Revenue
2022,3.554729,32.332507,32.225978,0.006605,0.027724,0.009579,0.024723
2023,3.395105,28.197341,29.606277,0.007054,0.026530,0.009645,0.025812
2024,3.849670,30.189694,31.442808,0.006815,0.028410,0.009829,0.024405
2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2028,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2029,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Saved Net Working Capital table as 'BJ_Net_Working_Capital_Table.csv'.


In [ ]:


# 1. Update FCF Table with Current Assets and Current Liabilities from NWC Table
fcf_df["Current Assets"] = nwc_df["Current Assets"]
fcf_df["Current Liabilities"] = nwc_df["Current Liabilities"]

# 2. Calculate NWC and Change in NWC for All Years
# NWC = Current Assets - Current Liabilities
fcf_df["NWC"] = fcf_df["Current Assets"] - fcf_df["Current Liabilities"]

# Change in NWC = This Year NWC - Prev Year NWC
fcf_df["Change in NWC"] = fcf_df["NWC"].diff()
fcf_df.loc[2023, "Change in NWC"] = 0  # No prior year for 2023, set to 0

# 3. Update Estimated Years (2026-2030) with Remaining Values
for year in estimate_years:
    # D&A from Fixed Assets Table
    fcf_df.loc[year, "D&A"] = fixed_assets_df.loc[year, "D&A"]

    # Operating Profit (EBIT) = EBITDA - D&A
    fcf_df.loc[year, "EBIT"] = fcf_df.loc[year, "EBITDA"] - fcf_df.loc[year, "D&A"]

    # Operating Taxes = Tax% of EBIT * EBIT
    avg_tax_pct = fcf_df.loc[given_years, "Tax% of EBIT"].mean()  # Use average from given years
    fcf_df.loc[year, "Operating Taxes"] = fcf_df.loc[year, "EBIT"] * avg_tax_pct

    # NOPAT = EBIT - Operating Taxes
    fcf_df.loc[year, "NOPAT"] = fcf_df.loc[year, "EBIT"] - fcf_df.loc[year, "Operating Taxes"]

# 4. Fetch CapEx from Fixed Assets Table for All Years
fcf_df["CapEx"] = fixed_assets_df["CapEx"]

# 5. Calculate Free Cash Flow for All Years
fcf_df["Free Cash Flow"] = (
    fcf_df["NOPAT"] +
    fcf_df["D&A"] -
    fcf_df["CapEx"] -
    fcf_df["Change in NWC"]
)

# Display the Updated FCF Table
print("Updated Free Cash Flow Table (in millions):")
display(fcf_df[["Revenue", "COGS", "Gross Profit", "SG&A", "Total Operating Expenses", "EBITDA",
                "D&A", "EBIT", "Operating Taxes", "NOPAT", "NWC", "Change in NWC", "CapEx", "Free Cash Flow"]])

# Save to CSV
fcf_df.to_csv("BJ_Free_Cash_Flow_Table_Updated.csv")
print("\nSaved updated Free Cash Flow table as 'BJ_Free_Cash_Flow_Table_Updated.csv'.")

Updated Free Cash Flow Table (in millions):


,Revenue,COGS,Gross Profit,SG&A,Total Operating Expenses,EBITDA,D&A,EBIT,Operating Taxes,NOPAT,NWC,Change in NWC,CapEx,Free Cash Flow
2022,226954.000000,199382.000000,27572.000000,19779.000000,19779.000000,7793.000000,1900.000000,7793.0,1949.850000,6156.470000,-10278.000000,NaN,3891.000000,NaN
2023,242290.000000,212586.000000,29704.000000,21590.000000,21590.000000,8114.000000,2077.000000,8114.0,1949.850000,6410.060000,-11857.000000,0.000000,4323.000000,4164.060000
2024,254453.000000,222358.000000,32095.000000,22810.000000,22810.000000,9285.000000,2237.000000,9285.0,1949.850000,7335.150000,-12259.000000,-402.000000,4710.000000,5264.150000
2025,268867.205857,235687.435459,33179.770397,23830.703018,23830.703018,9349.067380,2361.676130,6987.391249,1631.584248,5355.807002,-12763.008515,-504.008515,6721.680146,1499.811501
2026,284097.944945,249038.613133,35059.331813,25180.660216,25180.660216,9878.671597,2687.023453,7191.648144,1679.279062,5512.369081,-13486.005029,-722.996514,7102.448624,1819.940425
2027,300191.472087,263146.105817,37045.366271,26607.089537,26607.089537,10438.276734,3016.506351,7421.770383,1733.013540,5688.756843,-14249.957715,-763.952685,7504.786802,1964.429077
2028,317196.662339,278052.757102,39143.905236,28114.322958,28114.322958,11029.582278,3351.425774,7678.156505,1792.880741,5885.275763,-15057.186649,-807.228934,7929.916558,2114.013912
2029,335165.159421,293803.837577,41361.321844,29706.937856,29706.937856,11654.383988,3693.076770,7961.307217,1858.997583,6102.309634,-15910.143336,-852.956687,8379.128986,2269.214106



Saved updated Free Cash Flow table as 'BJ_Free_Cash_Flow_Table_Updated.csv'.


THE COSTCO WHOLESALE EXAMPLE OF CORRECT IMPLIED SHARE PRICE

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# Define the ticker
ticker = "COST"
cost = yf.Ticker(ticker)

# Financial statement data (from your input, in millions USD)
income_stmt_data = {
    2019: {
        "Total Revenue": 152703, "Cost Of Revenue": 132886, "Selling General And Administration": 13502,
        "Depreciation & Amortization": 1492, "Operating Income": 4737, "Interest Expense": -150,
        "Tax Provision": 1061, "Net Income": 3704
    },
    2020: {
        "Total Revenue": 166761, "Cost Of Revenue": 144939, "Selling General And Administration": 14687,
        "Depreciation & Amortization": 1645, "Operating Income": 5435, "Interest Expense": -160,
        "Tax Provision": 1308, "Net Income": 4059
    },
    2021: {
        "Total Revenue": 195929, "Cost Of Revenue": 170684, "Selling General And Administration": 16680,
        "Depreciation & Amortization": 1781, "Operating Income": 6708, "Interest Expense": -171,
        "Tax Provision": 1601, "Net Income": 5079
    }
}

balance_sheet_data = {
    2018: {
        "Cash And Cash Equivalents": 6055, "Receivables": 1669, "Merchandise Inventories": 11040,
        "Other Current Assets": 321, "Net PPE": 19681, "Accounts Payable": 11237,
        "Accrued Salaries And Benefits": 2994, "Accrued Member Rewards": 1057,
        "Deferred Membership Fees": 1624, "Current Portion of Long Term Debt": 90,
        "Other Current Liabilities": 2924, "Long Term Debt": 6487, "Share Issued": 441.825
    },
    2019: {
        "Cash And Cash Equivalents": 8384, "Receivables": 1535, "Merchandise Inventories": 11395,
        "Other Current Assets": 1111, "Net PPE": 20890, "Accounts Payable": 11679,
        "Accrued Salaries And Benefits": 3176, "Accrued Member Rewards": 1180,
        "Deferred Membership Fees": 1711, "Current Portion of Long Term Debt": 1699,
        "Other Current Liabilities": 3792, "Long Term Debt": 5124, "Share Issued": 441.255,
        "Noncontrolling Interests": 341
    },
    2020: {
        "Cash And Cash Equivalents": 12277, "Receivables": 1550, "Merchandise Inventories": 12242,
        "Other Current Assets": 1023, "Net PPE": 21807, "Accounts Payable": 14172,
        "Accrued Salaries And Benefits": 3605, "Accrued Member Rewards": 1393,
        "Deferred Membership Fees": 1851, "Current Portion of Long Term Debt": 95,
        "Other Current Liabilities": 3728, "Long Term Debt": 7514, "Share Issued": 441.255,
        "Noncontrolling Interests": 421
    },
    2021: {
        "Cash And Cash Equivalents": 11258, "Receivables": 1803, "Merchandise Inventories": 14215,
        "Other Current Assets": 1312, "Net PPE": 23492, "Accounts Payable": 16278,
        "Accrued Salaries And Benefits": 4090, "Accrued Member Rewards": 1671,
        "Deferred Membership Fees": 2042, "Current Portion of Long Term Debt": 799,
        "Other Current Liabilities": 4561, "Long Term Debt": 6692, "Share Issued": 441.255,
        "Noncontrolling Interests": 514
    }
}

cash_flow_data = {
    2019: {
        "Depreciation & Amortization": 1492, "Capital Expenditure": -2998,
        "Change In Receivables": np.nan, "Change In Inventory": -536, "Change In Accounts Payable": 322,
        "Other Operating Activities": 623
    },
    2020: {
        "Depreciation & Amortization": 1645, "Capital Expenditure": -2810,
        "Change In Receivables": np.nan, "Change In Inventory": -791, "Change In Accounts Payable": 2261,
        "Other Operating Activities": 728
    },
    2021: {
        "Depreciation & Amortization": 1781, "Capital Expenditure": -3588,
        "Change In Receivables": np.nan, "Change In Inventory": -1892, "Change In Accounts Payable": 1838,
        "Other Operating Activities": 1057
    }
}

# Convert to DataFrames
income_stmt_df = pd.DataFrame(income_stmt_data).T
balance_sheet_df = pd.DataFrame(balance_sheet_data).T
cash_flow_df = pd.DataFrame(cash_flow_data).T

# Define years
given_years = [2019, 2020, 2021]
estimate_years = [2022, 2023, 2024, 2025, 2026]
all_years = given_years + estimate_years
latest_year = given_years[-1]  # 2021
last_estimate_year = estimate_years[-1]  # 2026

# --- Free Cash Flow Table ---
fcf_df = pd.DataFrame(index=all_years)

# Given Years (2019-2021)
for year in given_years:
    fcf_df.loc[year, "Revenue"] = income_stmt_df.loc[year, "Total Revenue"]
    fcf_df.loc[year, "COGS"] = income_stmt_df.loc[year, "Cost Of Revenue"]
    fcf_df.loc[year, "Gross Profit"] = fcf_df.loc[year, "Revenue"] - fcf_df.loc[year, "COGS"]
    fcf_df.loc[year, "SG&A"] = income_stmt_df.loc[year, "Selling General And Administration"]
    fcf_df.loc[year, "Total Operating Expenses"] = fcf_df.loc[year, "SG&A"]
    fcf_df.loc[year, "EBITDA"] = fcf_df.loc[year, "Gross Profit"] - fcf_df.loc[year, "Total Operating Expenses"]
    fcf_df.loc[year, "D&A"] = income_stmt_df.loc[year, "Depreciation & Amortization"]
    fcf_df.loc[year, "EBIT"] = fcf_df.loc[year, "EBITDA"] - fcf_df.loc[year, "D&A"]
    fcf_df.loc[year, "Operating Taxes"] = income_stmt_df.loc[year, "Tax Provision"]
    fcf_df.loc[year, "NOPAT"] = fcf_df.loc[year, "EBIT"] - fcf_df.loc[year, "Operating Taxes"]
    # NWC
    current_assets = (
        balance_sheet_df.loc[year, "Receivables"] +
        balance_sheet_df.loc[year, "Merchandise Inventories"] +
        balance_sheet_df.loc[year, "Other Current Assets"]
    )
    current_liabilities = (
        balance_sheet_df.loc[year, "Accounts Payable"] +
        balance_sheet_df.loc[year, "Accrued Salaries And Benefits"] +
        balance_sheet_df.loc[year, "Accrued Member Rewards"] +
        balance_sheet_df.loc[year, "Deferred Membership Fees"] +
        balance_sheet_df.loc[year, "Other Current Liabilities"]
    )
    fcf_df.loc[year, "NWC"] = current_assets - current_liabilities
    # Change in NWC
    fcf_df.loc[year, "Change in NWC"] = (
        fcf_df.loc[year, "NWC"] - fcf_df.loc[year-1, "NWC"] if year != given_years[0]
        else 0
    )

# Assumptions for Given Years
fcf_df["Revenue Growth"] = fcf_df["Revenue"].pct_change()
fcf_df["COGS% of Revenue"] = fcf_df["COGS"] / fcf_df["Revenue"]
fcf_df["SG&A% of Revenue"] = fcf_df["SG&A"] / fcf_df["Revenue"]
fcf_df["Tax% of EBIT"] = fcf_df["Operating Taxes"] / fcf_df["EBIT"]

# Calculate averages
avg_revenue_growth = fcf_df.loc[given_years, "Revenue Growth"].mean()
avg_cogs_pct = fcf_df.loc[given_years, "COGS% of Revenue"].mean()
avg_sga_pct = fcf_df.loc[given_years, "SG&A% of Revenue"].mean()
avg_tax_pct = fcf_df.loc[given_years, "Tax% of EBIT"].mean()

# Adjust revenue growth for estimates (mean ± 2%)
estimate_revenue_growth = avg_revenue_growth  # Example: use mean

# --- Fixed Assets Table ---
fixed_assets_df = pd.DataFrame(index=all_years)

# Given Years (2019-2021)
for year in given_years:
    fixed_assets_df.loc[year, "Beginning PPE"] = (
        balance_sheet_df.loc[year-1, "Net PPE"] if year != 2019
        else balance_sheet_df.loc[2018, "Net PPE"]
    )
    fixed_assets_df.loc[year, "D&A"] = income_stmt_df.loc[year, "Depreciation & Amortization"]
    fixed_assets_df.loc[year, "Ending PPE"] = balance_sheet_df.loc[year, "Net PPE"]
    fixed_assets_df.loc[year, "CapEx"] = (
        fixed_assets_df.loc[year, "Ending PPE"] -
        fixed_assets_df.loc[year, "Beginning PPE"] +
        fixed_assets_df.loc[year, "D&A"]
    )

# Assumptions for Fixed Assets
fixed_assets_df["D&A% of Beginning PPE"] = fixed_assets_df["D&A"] / fixed_assets_df["Beginning PPE"]
fixed_assets_df["CapEx% of Beginning PPE"] = fixed_assets_df["CapEx"] / fixed_assets_df["Beginning PPE"]

# Calculate averages
avg_da_pct = fixed_assets_df.loc[given_years, "D&A% of Beginning PPE"].mean()
avg_capex_pct = fixed_assets_df.loc[given_years, "CapEx% of Beginning PPE"].mean()

# --- Net Working Capital Table ---
nwc_df = pd.DataFrame(index=all_years)

# Given Years (2019-2021)
for year in given_years:
    nwc_df.loc[year, "Accounts Receivable"] = balance_sheet_df.loc[year, "Receivables"]
    nwc_df.loc[year, "Merchandise Inventory"] = balance_sheet_df.loc[year, "Merchandise Inventories"]
    nwc_df.loc[year, "Other Current Assets"] = balance_sheet_df.loc[year, "Other Current Assets"]
    nwc_df.loc[year, "Accounts Payable"] = balance_sheet_df.loc[year, "Accounts Payable"]
    nwc_df.loc[year, "Current Accrued Expenses"] = balance_sheet_df.loc[year, "Accrued Salaries And Benefits"]
    nwc_df.loc[year, "Deferred Membership Fees"] = balance_sheet_df.loc[year, "Deferred Membership Fees"]
    nwc_df.loc[year, "Other Current Liabilities"] = balance_sheet_df.loc[year, "Other Current Liabilities"]

# Assumptions for NWC
nwc_df["DSO"] = (nwc_df["Accounts Receivable"] / fcf_df["Revenue"]) * 360
nwc_df["DIO"] = (nwc_df["Merchandise Inventory"] / fcf_df["COGS"]) * 360
nwc_df["DPO"] = (nwc_df["Accounts Payable"] / fcf_df["COGS"]) * 360
nwc_df["Other Current Assets% of Revenue"] = nwc_df["Other Current Assets"] / fcf_df["Revenue"]
nwc_df["Current Accrued Expenses% of Revenue"] = nwc_df["Current Accrued Expenses"] / fcf_df["Revenue"]
nwc_df["Deferred Membership Fees% of Revenue"] = nwc_df["Deferred Membership Fees"] / fcf_df["Revenue"]
nwc_df["Other Current Liabilities% of Revenue"] = nwc_df["Other Current Liabilities"] / fcf_df["Revenue"]

# Calculate averages
avg_dso = nwc_df.loc[given_years, "DSO"].mean()
avg_dio = nwc_df.loc[given_years, "DIO"].mean()
avg_dpo = nwc_df.loc[given_years, "DPO"].mean()
avg_other_ca_pct = nwc_df.loc[given_years, "Other Current Assets% of Revenue"].mean()
avg_accrued_exp_pct = nwc_df.loc[given_years, "Current Accrued Expenses% of Revenue"].mean()
avg_deferred_fees_pct = nwc_df.loc[given_years, "Deferred Membership Fees% of Revenue"].mean()
avg_other_cl_pct = nwc_df.loc[given_years, "Other Current Liabilities% of Revenue"].mean()

# Estimate Years (2022-2026)
for year in estimate_years:
    # FCF Table
    fcf_df.loc[year, "Revenue"] = fcf_df.loc[year-1, "Revenue"] * (1 + estimate_revenue_growth)
    fcf_df.loc[year, "COGS"] = fcf_df.loc[year, "Revenue"] * avg_cogs_pct
    fcf_df.loc[year, "Gross Profit"] = fcf_df.loc[year, "Revenue"] - fcf_df.loc[year, "COGS"]
    fcf_df.loc[year, "SG&A"] = fcf_df.loc[year, "Revenue"] * avg_sga_pct
    fcf_df.loc[year, "Total Operating Expenses"] = fcf_df.loc[year, "SG&A"]
    fcf_df.loc[year, "EBITDA"] = fcf_df.loc[year, "Gross Profit"] - fcf_df.loc[year, "Total Operating Expenses"]
    # Fixed Assets
    fixed_assets_df.loc[year, "Beginning PPE"] = fixed_assets_df.loc[year-1, "Ending PPE"]
    fixed_assets_df.loc[year, "D&A"] = fixed_assets_df.loc[year, "Beginning PPE"] * avg_da_pct
    fixed_assets_df.loc[year, "CapEx"] = fixed_assets_df.loc[year, "Beginning PPE"] * avg_capex_pct
    fixed_assets_df.loc[year, "Ending PPE"] = (
        fixed_assets_df.loc[year, "Beginning PPE"] -
        fixed_assets_df.loc[year, "D&A"] +
        fixed_assets_df.loc[year, "CapEx"]
    )
    # Populate FCF with D&A and CapEx
    fcf_df.loc[year, "D&A"] = fixed_assets_df.loc[year, "D&A"]
    fcf_df.loc[year, "EBIT"] = fcf_df.loc[year, "EBITDA"] - fcf_df.loc[year, "D&A"]
    fcf_df.loc[year, "Operating Taxes"] = fcf_df.loc[year, "EBIT"] * avg_tax_pct
    fcf_df.loc[year, "NOPAT"] = fcf_df.loc[year, "EBIT"] - fcf_df.loc[year, "Operating Taxes"]
    # NWC Table
    nwc_df.loc[year, "Accounts Receivable"] = (avg_dso * fcf_df.loc[year, "Revenue"]) / 360
    nwc_df.loc[year, "Merchandise Inventory"] = (avg_dio * fcf_df.loc[year, "COGS"]) / 360
    nwc_df.loc[year, "Other Current Assets"] = fcf_df.loc[year, "Revenue"] * avg_other_ca_pct
    nwc_df.loc[year, "Accounts Payable"] = (avg_dpo * fcf_df.loc[year, "COGS"]) / 360
    nwc_df.loc[year, "Current Accrued Expenses"] = fcf_df.loc[year, "Revenue"] * avg_accrued_exp_pct
    nwc_df.loc[year, "Deferred Membership Fees"] = fcf_df.loc[year, "Revenue"] * avg_deferred_fees_pct
    nwc_df.loc[year, "Other Current Liabilities"] = fcf_df.loc[year, "Revenue"] * avg_other_cl_pct
    # Calculate NWC
    current_assets = (
        nwc_df.loc[year, "Accounts Receivable"] +
        nwc_df.loc[year, "Merchandise Inventory"] +
        nwc_df.loc[year, "Other Current Assets"]
    )
    current_liabilities = (
        nwc_df.loc[year, "Accounts Payable"] +
        nwc_df.loc[year, "Current Accrued Expenses"] +
        nwc_df.loc[year, "Deferred Membership Fees"] +
        nwc_df.loc[year, "Other Current Liabilities"]
    )
    fcf_df.loc[year, "NWC"] = current_assets - current_liabilities
    fcf_df.loc[year, "Change in NWC"] = (
        fcf_df.loc[year, "NWC"] - fcf_df.loc[year-1, "NWC"]
    )
    # CapEx
    fcf_df.loc[year, "CapEx"] = fixed_assets_df.loc[year, "CapEx"]
    # Free Cash Flow
    fcf_df.loc[year, "Free Cash Flow"] = (
        fcf_df.loc[year, "NOPAT"] +
        fcf_df.loc[year, "D&A"] -
        fcf_df.loc[year, "CapEx"] -
        fcf_df.loc[year, "Change in NWC"]
    )

# --- WACC Table ---
wacc_df = pd.DataFrame(index=[latest_year])

wacc_df["Equity"] = cost.info["marketCap"] / 1_000_000
wacc_df["Debt"] = (
    balance_sheet_df.loc[latest_year, "Long Term Debt"] +
    balance_sheet_df.loc[latest_year, "Current Portion of Long Term Debt"]
)
interest_expense_2019 = abs(income_stmt_df.loc[2019, "Interest Expense"])
wacc_df["Cost of Debt"] = interest_expense_2019 / wacc_df["Debt"]
wacc_df["D/D+E"] = wacc_df["Debt"] / (wacc_df["Debt"] + wacc_df["Equity"])
wacc_df["Tax Rate"] = 0.21
wacc_df["After Tax Cost of Debt"] = wacc_df["Cost of Debt"] * (1 - wacc_df["Tax Rate"])
wacc_df["Risk Free Rate"] = 0.0429
wacc_df["Expected Market Return"] = 0.10  # Adjusted to 10% (3.9% gives negative premium)
wacc_df["Market Risk Premium"] = wacc_df["Expected Market Return"] - wacc_df["Risk Free Rate"]
wacc_df["Beta"] = cost.info.get("beta", 0.94)
wacc_df["E/D+E"] = wacc_df["Equity"] / (wacc_df["Debt"] + wacc_df["Equity"])
wacc_df["Cost of Equity"] = wacc_df["Risk Free Rate"] + wacc_df["Beta"] * wacc_df["Market Risk Premium"]
wacc_df["WACC"] = (
    wacc_df["After Tax Cost of Debt"] * wacc_df["D/D+E"] +
    wacc_df["Cost of Equity"] * wacc_df["E/D+E"]
)

# --- DCF Table ---
dcf_df = pd.DataFrame(index=estimate_years)
dcf_df["PY"] = range(1, 6)
dcf_df["Free Cash Flow"] = fcf_df.loc[estimate_years, "Free Cash Flow"]
wacc = wacc_df.loc[latest_year, "WACC"]
dcf_df["PVFCF"] = dcf_df["Free Cash Flow"] / (1 + wacc) ** dcf_df["PY"]

# Growth Rate = 2.5% (7.3% > WACC, adjusted)
growth_rate = 0.025
fcf_last_year = dcf_df.loc[last_estimate_year, "Free Cash Flow"]
tv = (fcf_last_year * (1 + growth_rate)) / (wacc - growth_rate) if wacc > growth_rate else np.inf
dcf_df.loc[last_estimate_year, "Terminal Value"] = tv
dcf_df.loc[last_estimate_year, "PV of TV"] = tv / (1 + wacc) ** 5
sum_pvfcf = dcf_df["PVFCF"].sum()
dcf_df.loc[last_estimate_year, "Sum of PVFCF"] = sum_pvfcf
ev = dcf_df.loc[last_estimate_year, "PV of TV"] + sum_pvfcf
dcf_df.loc[last_estimate_year, "EV"] = ev
cash = balance_sheet_df.loc[latest_year, "Cash And Cash Equivalents"]
debt = wacc_df.loc[latest_year, "Debt"]
minority_interest = balance_sheet_df.loc[latest_year, "Noncontrolling Interests"]
equity_value = ev + cash - debt - minority_interest
dcf_df.loc[last_estimate_year, "Equity Value"] = equity_value
diluted_shares = balance_sheet_df.loc[latest_year, "Share Issued"]
dcf_df.loc[last_estimate_year, "Implied Share Price"] = equity_value / diluted_shares

# Display Tables
print("Free Cash Flow Table (in millions):")
display(fcf_df)
print("\nFixed Assets Table (in millions):")
display(fixed_assets_df)
print("\nNet Working Capital Table (in millions):")
display(nwc_df)
print("\nWACC Table (in millions for Equity and Debt, percentages for rates):")
display(wacc_df)
print("\nDCF Table (in millions except Implied Share Price):")
display(dcf_df)

# Save to CSV
fcf_df.to_csv("COST_FCF_Table.csv")
fixed_assets_df.to_csv("COST_Fixed_Assets_Table.csv")
nwc_df.to_csv("COST_NWC_Table.csv")
wacc_df.to_csv("COST_WACC_Table.csv")
dcf_df.to_csv("COST_DCF_Table.csv")
print("\nSaved tables as CSV files.")

<ipython-input-1-fe47ec229be1>:128: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fcf_df["Revenue Growth"] = fcf_df["Revenue"].pct_change()


Free Cash Flow Table (in millions):


,Revenue,COGS,Gross Profit,SG&A,Total Operating Expenses,EBITDA,D&A,EBIT,Operating Taxes,NOPAT,NWC,Change in NWC,Revenue Growth,COGS% of Revenue,SG&A% of Revenue,Tax% of EBIT,CapEx,Free Cash Flow
2019,152703.000000,132886.000000,19817.000000,13502.000000,13502.000000,6315.000000,1492.000000,4823.000000,1061.000000,3762.000000,-7497.000000,0.000000,NaN,0.870225,0.088420,0.219988,NaN,NaN
2020,166761.000000,144939.000000,21822.000000,14687.000000,14687.000000,7135.000000,1645.000000,5490.000000,1308.000000,4182.000000,-9934.000000,-2437.000000,0.092061,0.869142,0.088072,0.238251,NaN,NaN
2021,195929.000000,170684.000000,25245.000000,16680.000000,16680.000000,8565.000000,1781.000000,6784.000000,1601.000000,5183.000000,-11312.000000,-1378.000000,0.174909,0.871152,0.085133,0.235996,NaN,NaN
2022,222082.588561,193250.315211,28832.273350,19367.455059,19367.455059,9464.818291,1849.806990,7615.011302,1762.203442,5852.807860,-10496.606270,815.393730,0.000000,NaN,NaN,NaN,3279.648929,3607.572190
2023,251727.289692,219046.339451,32680.950241,21952.720391,21952.720391,10728.229850,1962.395599,8765.834251,2028.517448,6737.316804,-11897.746079,-1401.139809,0.000000,NaN,NaN,NaN,3479.264950,6621.587262
2024,285329.114661,248285.746777,37043.367884,24883.079944,24883.079944,12160.287940,2081.836921,10078.451020,2332.272451,7746.178568,-13485.917078,-1588.170999,0.000000,NaN,NaN,NaN,3691.030613,7725.155876
2025,323416.280264,281428.177284,41988.102980,28204.598632,28204.598632,13783.504348,2208.548045,11574.956303,2678.581427,8896.374876,-15286.085132,-1800.168054,0.000000,NaN,NaN,NaN,3915.685405,8989.405571
2026,366587.512333,318994.626142,47592.886191,31969.490343,31969.490343,15623.395848,2342.971450,13280.424398,3073.246862,10207.177537,-17326.548674,-2040.463542,0.000000,NaN,NaN,NaN,4154.013823,10436.598705



Fixed Assets Table (in millions):


,Beginning PPE,D&A,Ending PPE,CapEx,D&A% of Beginning PPE,CapEx% of Beginning PPE
2019,19681.000000,1492.000000,20890.000000,2701.000000,0.075809,0.137239
2020,20890.000000,1645.000000,21807.000000,2562.000000,0.078746,0.122642
2021,21807.000000,1781.000000,23492.000000,3466.000000,0.081671,0.158940
2022,23492.000000,1849.806990,24921.841939,3279.648929,NaN,NaN
2023,24921.841939,1962.395599,26438.711290,3479.264950,NaN,NaN
2024,26438.711290,2081.836921,28047.904983,3691.030613,NaN,NaN
2025,28047.904983,2208.548045,29755.042342,3915.685405,NaN,NaN
2026,29755.042342,2342.971450,31566.084715,4154.013823,NaN,NaN


...


WACC Table (in millions for Equity and Debt, percentages for rates):


,Equity,Debt,Cost of Debt,D/D+E,Tax Rate,After Tax Cost of Debt,Risk Free Rate,Expected Market Return,Market Risk Premium,Beta,E/D+E,Cost of Equity,WACC
2021,427448.631296,7491.0,0.020024,0.017223,0.21,0.015819,0.0429,0.1,0.0571,0.939,0.982777,0.096517,0.095127



DCF Table (in millions except Implied Share Price):


,PY,Free Cash Flow,PVFCF,Terminal Value,PV of TV,Sum of PVFCF,EV,Equity Value,Implied Share Price
2022,1,3607.572190,3294.204308,NaN,NaN,NaN,NaN,NaN,NaN
2023,2,6621.587262,5521.195271,NaN,NaN,NaN,NaN,NaN,NaN
2024,3,7725.155876,5881.847390,NaN,NaN,NaN,NaN,NaN,NaN
2025,4,8989.405571,6249.898667,NaN,NaN,NaN,NaN,NaN,NaN
2026,5,10436.598705,6625.771986,152544.791427,96844.482979,27572.917621,124417.4006,127670.4006,289.33474



Saved tables as CSV files.
